# Walmart sales

Walmart's marketing service is looking for a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made. 

Supervised Machine learning will be used in this project. The target variable is quantitative, thus it is a regression will be used.

The applied model here is a linear regression model.

# Content
- 1- EDA and preprocessing
    - 1-1 basic statistics
    - 1-2 pandas preprocessing
    - 1-3 EDA
    - 1-4 Preprocessing with scikit-learn
- 2- Baseline model
    - 2-1 model
    - 2-2 Performance of the model
- 3- Regularized linear regression
    - 3-1 Ridge model score
    - 3-2 Lasso model score
- Conclusion 
    

In [1]:
import pandas as pd
from datetime import datetime

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso

Let's load the data by including a dateparse to be able to use a date format later on in the bookmark

In [2]:
dateparse = lambda x: pd.to_datetime(x, format = '%d-%m-%Y', errors='coerce')

dataset = pd.read_csv('Walmart_Store_sales.csv', parse_dates=['Date'], date_parser=dateparse)

# 1- EDA and preprocessing

## 1-1 basic statistics

In [3]:
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all', datetime_is_numeric=True) #datetime_is_numeric to avoid depreciation warning
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

print("Data types")
display(dataset.dtypes)

Number of rows : 150

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,2012-07-27,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
mean,9.866667,2011-05-07 09:05:27.272727296,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
min,1.000000,2010-02-05 00:00:00,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,2010-08-16 12:00:00,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,2011-05-09 12:00:00,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,2012-01-14 18:00:00,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000
max,20.000000,2012-10-19 00:00:00,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

Data types


Store                  float64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag           float64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object

## 1-2 Pandas preprocessing

Pandas preprocessing to be done:

**Raws to drop:**
- Weekly_Sales where data is NaN ( as Weekly_Sales is the target)
- outliers in Temperature, Fuel_Price, CPI, Unemployment

**Rework on columns:**
- 'Date' column needs to be transformed to date format
- Need to create Year/Month/day/Day_of_week to use the data properly

**column to drop:**
- date, after reshaping per above

**ML Preprocessing:**
- SimpleImputer to manage missing values
- OneHotEncoder for categorical variables (Store, Holiday_Flag)
- Standardization for numeric features (Temperature, Fuel_Price, CPI, Unemplyment, Year, Month, Day, DayOfWeek)

### Weekly_Sales columns: treatment of the NaN
- As weekly_Sales is the target, the rows where Weekly_sales is missing need to be removed

Let's first check the number of raws to be dropped:

In [4]:
dataset['Weekly_Sales'].isna().sum()

14

then remove the raws where Weekly_Sales is NaN

In [5]:
dataset = dataset.dropna(subset=['Weekly_Sales']).reset_index(drop=True)

and check the new number of raws after the drop of raws

In [6]:
dataset.shape[0]

136

There are now 136 rows, as expected

### Date column: creation of new columns with 1 item per column

The Date column cannot be included as such in the model. Let's create new columns that contain year, month, day, day_of_week as numeric features

In [7]:
dataset['Year'] = dataset['Date'].dt.year  
dataset['Month'] = dataset['Date'].dt.month  
dataset['Day'] = dataset['Date'].dt.day
dataset['Day_of_week'] = dataset['Date'].dt.dayofweek

dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Day_of_week
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
2,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
3,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
4,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


### Remove outliers
Let's remove outliers on Temperatures, Fuel_Price, CPI, Unemployment (outlier = +/- 3std)

In [8]:
Temp_to_keep = (np.abs(dataset['Temperature']-dataset['Temperature'].mean()) <= (3*dataset['Temperature'].std()) ) | dataset['Temperature'].isnull()
Fuel_to_keep =(np.abs(dataset['Fuel_Price']-dataset['Fuel_Price'].mean()) <= (3*dataset['Fuel_Price'].std()) ) | dataset['Fuel_Price'].isnull()
CPI_to_keep = (np.abs(dataset['CPI']-dataset['CPI'].mean()) <= (3*dataset['CPI'].std()) ) | dataset['CPI'].isnull()
Unemp_to_keep = (np.abs(dataset['Unemployment']-dataset['Unemployment'].mean()) <= (3*dataset['Unemployment'].std()) ) | dataset['Unemployment'].isnull()

In [9]:
mask = (Temp_to_keep) & (Fuel_to_keep) & (CPI_to_keep) & (Unemp_to_keep)

In [10]:
filtered = dataset[mask].reset_index(drop = True)
# dropping column that is no longer necessary
df = filtered.drop(columns= ['Date'])
df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Day_of_week
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
2,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
3,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
4,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


### Final dataset for ML

In [11]:
print("Number of rows : {}".format(df.shape[0]))
print()

print("Display of dataset: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

print("Count of data types: ")
display(df.dtypes.value_counts())

Number of rows : 131

Display of dataset: 


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Day_of_week
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
2,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
3,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
4,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0



Basics statistics: 


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Day_of_week
count,131.000000,1.310000e+02,120.000000,117.000000,119.000000,120.000000,117.000000,113.000000,113.000000,113.000000,113.0
mean,9.938931,1.257990e+06,0.066667,60.405897,3.302908,180.175755,7.399427,2010.831858,6.274336,16.530973,4.0
std,6.228663,6.577463e+05,0.250490,18.466740,0.475435,39.723167,0.994117,0.822699,3.179869,8.238705,0.0
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000,2010.000000,1.000000,1.000000,4.0
25%,4.000000,5.842439e+05,0.000000,44.820000,2.824000,132.579257,6.664000,2010.000000,4.000000,10.000000,4.0
50%,9.000000,1.366396e+06,0.000000,61.790000,3.435000,197.655672,7.368000,2011.000000,6.000000,17.000000,4.0
75%,16.000000,1.809576e+06,0.000000,75.540000,3.708500,214.904838,8.099000,2012.000000,9.000000,24.000000,4.0
max,20.000000,2.771397e+06,1.000000,91.650000,4.170000,226.968844,9.524000,2012.000000,12.000000,31.000000,4.0



Percentage of missing values: 


Store            0.000000
Weekly_Sales     0.000000
Holiday_Flag     8.396947
Temperature     10.687023
Fuel_Price       9.160305
CPI              8.396947
Unemployment    10.687023
Year            13.740458
Month           13.740458
Day             13.740458
Day_of_week     13.740458
dtype: float64

Count of data types: 


float64    11
dtype: int64

- There are only few raws in this dataset; somehow running a machine learning model on such a few number of information may lead to misleading conslusions
- From 'Display of dataset' we see that the dataset is now structured with usable columns, especially on the periods of time
- In basic statistics we see that the outliers have been removed. There are no weird negative data. However Day_Of_Week has a single value at 4 => let's drop this column
- the percentage of missing values show some acceptable percentage of missing data, and will be treated in the next steps
- Data types show only float which is ok to apply machine learning model.

Let's then drop 'day_of_week' column and go ahead with EDA

In [12]:
df = df.drop(columns= ['Day_of_week'])

## 1-3 EDA

We are requested to apply a linear regression model. This means the following assumptions: linearity, homoscedacity, independance of residuals and non-colinearity of the independant variables.

Before going ahead we need to check if there is any dependancy or collinearity between the variables. In case there is collinearity we need to keep only 1 variable out of the ones showing dependencies. 

Let's first visualize if there are any pairwise dependencies

In [13]:
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
        autosize=False, height = 1200, width = 1200)
fig.show()

=> It looks like there is no dependancies.

Let's checks the correlation matrix

In [14]:
corr_matrix = df.corr().round(2)

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

- correlation vs weekly_sales (target):
CPI shows the highest negative correlation (-31%), followed by temperatures (-16%). The highest positive correlation with the target is Umployment at 19% (which makes sense since Walmart is a retail store with quite low prices). These correlations are however small.
- Correlation between the features: the highest correlation is between store and CPI, that remains acceptable to keep the 2 variables

Let's look at the distribution of variables

In [15]:
# Univariate analysis
# Distribution of each numeric variable
num_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year' , 'Month', 'Day']
for i in range(len(num_features)):
    fig = px.histogram(df[num_features[i]], nbins = 100)
    fig.show()

In [16]:
# Univariate analysis
# Barplot of each qualitative variable
cat_features = ['Store', 'Holiday_Flag']
for i in range(len(cat_features)):
    fig = px.bar(df[cat_features[i]])
    fig.show()

=> No specific trend is popping from these charts. Let's look at the feature vs target

## Looking at the target

In [17]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("weekly_sales vs temperature", "weekly_sales vs fuel price", "weekly_sales vs CPI", "weekly_sales vs Unemployment"))

fig.add_trace(go.Scatter(x=df['Weekly_Sales'], y=df['Temperature'], mode = "markers"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=df['Weekly_Sales'], y=df['Fuel_Price'], mode = "markers"),
              row=1, col=2)

fig.add_trace(go.Scatter(x=df['Weekly_Sales'], y=df['CPI'], mode = "markers"),
              row=2, col=1)

fig.add_trace(go.Scatter(x=df['Weekly_Sales'], y=df['Unemployment'], mode = "markers"),
              row=2, col=2)

fig.update_layout(title_text="Weekly_sales vs ...")

fig.show()

In [18]:
#Total sales per store
fig = px.bar(df, x="Weekly_Sales", y="Store", orientation='h', title = "Weekly_sales per store (total value)")
fig.show()

Holiday_flag and Year/ Month/ Day do not seem of much interest to be showed vs target.

There is a great difference in the weekly_sales of the given stores; most performing ones are 14, 13, 2, 1. Low performers are 9, 16, 5, 7 and 3.

## 1-4 Preprocessing with scikit-learn

- Separate target variable Y from features X

In [19]:
features_list = ["Store", "Holiday_Flag", "Temperature", "Fuel_Price", "CPI", "Unemployment", "Year", "Month", "Day"]
target_variable = "Weekly_Sales"

X = df.loc[:,features_list]
Y = df.loc[:,target_variable]

- Define numeric/ categorical features

In [20]:
numeric_features = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]
numeric_features2 = ["Year", "Month", "Day"] #Will get a different treatment for missing data than the ones above
categorical_features = ["Store", "Holiday_Flag"]

# 2- Baseline model

## 2-1 model preprocessing and training

- Division of dataset into train set & test set

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

- Pipelines

Creating pipeline for numeric features

In [22]:
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] # Names of numeric columns in X_train/X_test
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])

Creating pipeline for dates

In [23]:
numeric_features2 = ['Year', 'Month', 'Day'] # Names of numeric columns in X_train/X_test
numeric_transformer2 = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=1)), # missing values will be replaced by nearest neighbor
    ('scaler', StandardScaler())
])

Creating pipeline for categorical features

In [24]:
categorical_features = ["Store", "Holiday_Flag"] # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

Using ColumnTransformer to make a preprocessor object that describes all the treatments to be done

In [25]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('date', numeric_transformer2, numeric_features2),
        ('cat', categorical_transformer, categorical_features)
    ])

Preprocessing

In [26]:
# Preprocessings on train set
X_train = preprocessor.fit_transform(X_train)

# Preprocessings on test set
X_test = preprocessor.transform(X_test) #no fit, only transform

- train model

In [27]:
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression()

In [28]:
# Predictions on training set
Y_train_pred = regressor.predict(X_train)

# Predictions on test set
Y_test_pred = regressor.predict(X_test)

## 2-2 Performance of the model

In [29]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))
print("R2 difference (train - test) : ", r2_score(Y_train, Y_train_pred)-r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9742641264186495
R2 score on test set :  0.9351459232281667
R2 difference (train - test) :  0.03911820319048276


### Which difference train - test is meaningful?

In [30]:
scores = cross_val_score(regressor, X_train, Y_train, cv = 5)
avg = scores.mean()
std = scores.std()
print('cross-validated accuracy = {}\nstandard deviation : {}'.format(avg, std))

cross-validated accuracy = 0.9462074689059687
standard deviation : 0.01847826565762739


- The standard deviation is 0.0184, which is smaller than the difference in R2 score train - test of 0.039.
- The current model is thus slightly overfitting. We will try to regularize this later on.

### What are the drivers of the model?

Here are the coefficients from the model

In [31]:
regressor.coef_

array([  -38253.22066255,   -40351.1331935 ,    99273.79813789,
         -98443.23105297,   -20645.76610762,    64151.49129245,
         -34075.00727005,   400335.60179179, -1260369.30599488,
         657704.28243206, -1404045.18348156,   -24511.79883057,
        -879263.7480235 ,  -753006.02428708, -1275524.41091137,
         725104.35494296,    13742.80111321,   580137.63103376,
         729183.60530195,  -629644.49213273, -1101009.13773628,
        -635875.16692534,  -118264.87860272,   130041.14971573,
         377320.6820351 ,   -48026.14758205])

and the related columns:

In [32]:
column_names = [ ]
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    elif name == 'date': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)


Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'x0_2.0', 'x0_3.0', 'x0_4.0', 'x0_5.0', 'x0_6.0', 'x0_7.0', 'x0_8.0', 'x0_9.0', 'x0_10.0', 'x0_11.0', 'x0_13.0', 'x0_14.0', 'x0_15.0', 'x0_16.0', 'x0_17.0', 'x0_18.0', 'x0_19.0', 'x0_20.0', 'x1_1.0']


Creating a dataframe with the coefficients and the names of features

In [33]:
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])

# computing abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance.head()

,coefficients
x0_11.0,13742.801113
Year,20645.766108
x0_6.0,24511.798831
Day,34075.007270
Temperature,38253.220663


In [34]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120}, # to avoid cropping of column names
                  height = 600
                 )
fig.show()


- Some of the stores themselves have the highest coefficients; probably analysing their mix of sales, geographic situation or promotions would help us understand better the drivers for their performance.
- This model shows however that stores 5, 9, 3, 16, 7 drive the weekly sales down (these stores are actually the low performers flagged in the EDA part). Probably we should focus on how to improve their performance.
- CPI, unemployment, Fuel price, temperature play a minor role according to this model.


# 3- Regularized linear regression

As learnt above, the model is slightly overfitting.
- Let's see however if a regularization makes the model even better with Ridge and Lasso. Both methods are used to reduce ovefitting for regression. 

- Ridge reduces the coefficients (but does not put them to 0). This method is better than Lasso when the variables are considered as being important.
- Lasso can put some coefficients at 0, which is kind of feature selection. This method is better when **some** of the features are considered important.

In this case then Lasso is probably better

## 3-1 Ridge model score

In [35]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.0001, 0.01, 0.1, 0.5] # 0 corresponds to no regularization
}
gridsearch_ridge = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
gridsearch_ridge.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_ridge.best_params_)
print("Best R2 score : ", gridsearch_ridge.best_score_)


Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.0}
Best R2 score :  0.9462074689059687


- Ridge regularization best parameter is 0 => Ridge is not useful in this case (probably due to a small dataset)

Let's try Lasso

## 3-2 Lasso model score

In [36]:
# Perform grid search
print("Grid search...")
regressor = Lasso()
# Grid of values to be tested
params = {
    'alpha': [80, 82, 85, 86, 87, 90, 100] # 0 corresponds to no regularization
}
gridsearch_lasso = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
gridsearch_lasso.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_lasso.best_params_)
print("Best R2 score : ", gridsearch_lasso.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 80}
Best R2 score :  0.9462261357378076


In [37]:
# Print R^2 scores
print("R2 score on training set : ", gridsearch_lasso.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch_lasso.score(X_test, Y_test))
print("R2 difference (train - test) : ", gridsearch_lasso.score(X_train, Y_train)-gridsearch_lasso.score(X_test, Y_test))

R2 score on training set :  0.9742490729981934
R2 score on test set :  0.9369848073996451
R2 difference (train - test) :  0.03726426559854834


With a regularized model using Lasso we achieve to get a difference train - test of 0.04 that is still above the standard deviation (0.0178).

# Conclusion

This model has an r2 score over 97% on train set but is ovefitting. After Lasso regularization it is still slightly overfitting.

The most important features are the stores themselves.

To complete this model could be interesting to 
- cluster the stores (by level of weekly_sales for example),
- gather more information (more raws), by day_of_week, with product details, information on promotions or any other feature that could help understanding the store's revenue.